In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

#cd command needs '%' not '!', while other Unix commands need '!'
%cd ./drive/My\ Drive/summer\ vacation 3
!pwd
!ls -l

Mounted at /content/drive
/content/drive/My Drive/summer vacation 3
/content/drive/My Drive/summer vacation 3
total 1798
drwx------ 2 root root    4096 Aug  3 09:25  data
-rw------- 1 root root   17852 Aug  9 09:58  LOL_data_model.ipynb
-rw------- 1 root root   26835 Aug 12 09:36 'mushroom CNN.ipynb'
-rw------- 1 root root 1791244 Aug 12 09:32 '버섯데이터 전이학습.ipynb'


DataSet Class

In [3]:
# import libraries

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pandas
import matplotlib.pyplot as plt

Load Data

In [4]:
# 데이터 경로 설정
Mushrooms_dataset_file = '/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms'

In [5]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torch
from torchvision import transforms

class MushroomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        # Create class to index mapping
        self.classes = sorted(os.listdir(root_dir))
        self.classes_index = {cls: i for i, cls in enumerate(self.classes)}

        # Traverse root directory and collect image paths and labels
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            if os.path.isdir(cls_dir):
                for file_name in os.listdir(cls_dir):
                    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.image_files.append(os.path.join(cls_dir, file_name))
                        self.labels.append(self.classes_index[cls_name])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        img_path = self.image_files[index]
        label = self.labels[index]

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        target = torch.tensor(label, dtype=torch.long)  # Convert label to tensor

        return image, target

In [11]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # 잘린 이미지 로드 허용

import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 전체 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms', transform=transform)

# 클래스별로 데이터 분할
class_indices = {cls: [] for cls in dataset.classes}
for idx, (img, label) in enumerate(dataset):
    class_indices[dataset.classes[label]].append(idx)

train_indices = []
test_indices = []

# 각 클래스별로 75%-25% 분할
for cls, indices in class_indices.items():
  n_train = int(len(class_indices) * 0.75)
  train_indices.extend(class_indices[:n_train])
  test_indices.extend(class_indices[n_train:])

# Subset을 이용하여 Train/Test Dataset 생성
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# 클래스 인덱스 확인
classes = sorted(dataset.classes)
classes_index = {c: i for i, c in enumerate(classes)}

print(classes)
print(classes_index)

TypeError: unhashable type: 'slice'

256x256 크기로 변환, 회전 x

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# 변환 정의 (Image Transformation)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 전체 데이터셋 로드 (Load Entire Dataset)
dataset = datasets.ImageFolder(root=Mushrooms_dataset_file, transform=transform)

# 데이터셋 크기 설정 (Set the size of training and testing datasets)
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# 데이터셋 분할 (Split Dataset)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 데이터 로더 설정 (Set up DataLoaders)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# 클래스 인덱스 확인 (Check Class Indices)
classes = sorted(dataset.classes)
classes_index = {c: i for i, c in enumerate(classes)}

print(classes)
print(classes_index)

['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
{'Agaricus': 0, 'Amanita': 1, 'Boletus': 2, 'Cortinarius': 3, 'Entoloma': 4, 'Hygrocybe': 5, 'Lactarius': 6, 'Russula': 7, 'Suillus': 8}


임의의 각도로 회전(-30~30)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 변환 정의
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 데이터 로드
train_dataset = datasets.ImageFolder(root=Mushrooms_dataset_file, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = datasets.ImageFolder(root=Mushrooms_dataset_file, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

# 클래스 인덱스 확인
classes = sorted(train_dataset.classes)
classes_index = {c: i for i, c in enumerate(classes)}

print(classes)
print(classes_index)


3차원그림 Relu사용, BCELoss 사용, Adam

In [ ]:
# classifier class

class Classifier(nn.Module):

    def __init__(self):
        # initialise parent pytorch class
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(3*256*256, 100),
            nn.LeakyReLU(),

            nn.LayerNorm(100),

            nn.Linear(100, 1),
            nn.Sigmoid()
        )

        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, using simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.001)

        # counter and accumulator for progress
        self.counter = 0
        self.progress = []

        pass


    def forward(self, inputs):
        # simply run model
        inputs = inputs.view(inputs.size(0), -1)
        return self.model(inputs)


    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)

        targets = targets.float()

        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 10000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, and update the weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass


    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5))
        pass

    def evaluate(self, data_loader):
        self.eval()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for inputs, targets in data_loader:
                outputs = self.forward(inputs)
                loss = self.loss_function(outputs, targets)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                _, actual = torch.max(targets, 1)
                correct_predictions += (predicted == actual).sum().item()
                total_samples += targets.size(0)

        avg_loss = total_loss / len(data_loader)
        accuracy = (correct_predictions / total_samples) * 100
        return avg_loss, accuracy

    pass

3차원, Relu사용, CrossEntropyLoss, Adam

In [ ]:
# classifier class

class Classifier(nn.Module):

    def __init__(self):
        # initialise parent pytorch class
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(3*224*224, 1000),
            nn.LeakyReLU(),

            nn.LayerNorm(1000),

            nn.Linear(1000, 10),
            nn.Sigmoid()
        )

        # create loss function
        self.loss_function = nn.CrossEntropyLoss()

        # create optimiser, using simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.001)

        # counter and accumulator for progress
        self.counter = 0
        self.progress = []

        pass


    def forward(self, inputs):
        # simply run model
        inputs = inputs.view(inputs.size(0), -1)
        return self.model(inputs)


    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)

        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 1000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, and update the weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass


    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5))
        pass

    def evaluate(self, data_loader):
        self.eval()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for inputs, targets in data_loader:
                outputs = self.forward(inputs)
                loss = self.loss_function(outputs, targets)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                _, actual = torch.max(targets, 1)
                correct_predictions += (predicted == actual).sum().item()
                total_samples += targets.size(0)

        avg_loss = total_loss / len(data_loader)
        accuracy = (correct_predictions / total_samples) * 100
        return avg_loss, accuracy

    pass

학습시키기

In [ ]:
%%time
# create neural network

C = Classifier()

# train neural network
epochs = 4

for i in range(epochs):
    print('training epoch', i+1, "of", epochs)
    for inputs, targets in train_loader:
        C.train(inputs, targets)
    # 평가
    train_loss, train_accuracy = C.evaluate(train_loader)
    test_loss, test_accuracy = C.evaluate(test_loader)

    # 결과 출력
    print(f'Epoch [{i + 1} / {epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


training epoch 1 of 4


KeyboardInterrupt: 